# Wizualizacja danych pogodowych

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sqlalchemy import create_engine, text
import os
from dotenv import load_dotenv
import plotly.express as px

## Przygotowanie danych z bazy

In [2]:
load_dotenv()

True

In [3]:
login = os.getenv("LOGIN")
password = os.getenv("PASSWORD")
db_host = os.getenv("DB_HOST")
port = os.getenv("PORT")
database = os.getenv("DB_NAME")

In [4]:
conn_str = (
    f"mssql+pyodbc://{login}:{password}@{db_host},{port}/{database}"
    "?driver=ODBC+Driver+17+for+SQL+Server"
)

In [5]:
engine = create_engine(conn_str)

In [ ]:
query_depesze = text("""
        SELECT Data,  KierunekWiatru, PredkoscWiatru, TemperaturaPowietrza
        FROM Synop.dbo.Depesze as depesze
        WHERE depesze.Data BETWEEN '2024-01-01' and '2024-12-31'
    """)

In [7]:


with engine.connect() as conn:
    df_depesze = pd.read_sql(query_depesze, conn)


In [15]:
df_srednie_dzienne = (
    df_depesze
    .groupby('Data', as_index=False)  # grupujemy po kolumnie Data (dzień)
    .agg({
        'TemperaturaPowietrza': 'mean',
        'PredkoscWiatru': 'mean',
    })
    .rename(columns={
        'TemperaturaPowietrza': 'SredniaTemp',
        'PredkoscWiatru': 'SredniaPredkoscWiatru'
    })
)


In [20]:
df_srednie_dzienne.head()

,Data,SredniaTemp,SredniaPredkoscWiatru,PrzedzialPredkosci
0,2024-01-01,3.033969,2.966808,2-5 m/s
1,2024-01-02,2.764544,3.519462,2-5 m/s
2,2024-01-03,4.085300,4.560424,2-5 m/s
3,2024-01-04,2.640575,4.896286,2-5 m/s
4,2024-01-05,-1.219833,3.192067,2-5 m/s


In [17]:
bins = [0, 2, 5, 8, 12, 17, 25, 35, 50]  # Przykładowe zakresy prędkości
labels = [f"{bins[i]}-{bins[i+1]} m/s" for i in range(len(bins)-1)]
df_srednie_dzienne['PrzedzialPredkosci'] = pd.cut(df_srednie_dzienne['SredniaPredkoscWiatru'], bins=bins, labels=labels, right=False)

In [22]:
fig = px.scatter(
    df_srednie_dzienne,
    x="Data",
    y="SredniaTemp",
    size="SredniaPredkoscWiatru",          # Rozmiar kropki pokazuje siłę wiatru
    color="PrzedzialPredkosci",           # Kolor kropki to kategoria prędkości (np. '2-5 m/s')
    hover_data=["SredniaPredkoscWiatru"], # Co wyświetlać dodatkowo w dymku
    title="Interaktywna wizualizacja dziennych danych pogodowych"
)

# Możesz dostosować zakres osi, format daty, układ itp.:
fig.update_layout(
    xaxis_title="Data",
    yaxis_title="Średnia Temperatura [°C]",
    legend_title="Prędkość wiatru",
    width=900,
    height=600
)

fig.show()

Ten interaktywny wykres przedstawia dzienną średnią temperaturę (oś pionowa) w funkcji daty (oś pozioma), z dodatkową informacją o wietrze:

- Rozmiar kropki odzwierciedla średnią prędkość wiatru danego dnia – im większa kropka, tym silniejszy wiatr.

- Kolor kropki to przedział prędkości wiatru (np. „0–2 m/s”, „2–5 m/s”, „5–8 m/s”, „8–12 m/s”).

Dzięki temu widzimy w jednym miejscu:

- Jak temperatura zmienia się w czasie (np. w przebiegu rocznym).

- Jakie wartości prędkości wiatru najczęściej występują w określonych miesiącach (np. zimą może być słabszy albo silniejszy, zależnie od danych).

- Które dni charakteryzowały się najbardziej wietrznymi warunkami (rozmiar i kolor punktów).

W dymkach (tooltipach) można podejrzeć dokładne wartości – datę, przedział prędkości wiatru, średnią temperaturę oraz średnią prędkość wiatru dla danego dnia.